<a href="https://colab.research.google.com/github/ParsaHejabi/ComputationalIntelligence-ComputerAssignments/blob/main/HW3/Stanford_Dogs_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Untar the Dataset from Google Drive

In [ ]:
!rm -r ./Images ./Annotation ./file_list.mat ./test_list.mat ./train_list.mat
!tar -xf ./drive/MyDrive/Stanford_Dogs_Dataset/images.tar
!tar -xf ./drive/MyDrive/Stanford_Dogs_Dataset/annotation.tar
!tar -xf ./drive/MyDrive/Stanford_Dogs_Dataset/lists.tar

# Importing libraries

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, random_split, DataLoader

import torchvision
from torchvision.utils import make_grid
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import torchvision.models as models

import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from PIL import Image
from collections import OrderedDict
from tqdm.notebook import tqdm

%matplotlib inline

# Preparing Dataset

## Load Images to PyTorch dataset format

In [ ]:
images = ImageFolder('./Images')

lenImages = len(images)
lenCategories = len(images.classes)

assert lenImages == 20580
assert lenCategories == 120

## Extract breed categories

In [ ]:
breeds = []
for breed in images.classes:
  breeds.append(breed.split('-')[1])

breeds

['Chihuahua',
 'Japanese_spaniel',
 'Maltese_dog',
 'Pekinese',
 'Shih',
 'Blenheim_spaniel',
 'papillon',
 'toy_terrier',
 'Rhodesian_ridgeback',
 'Afghan_hound',
 'basset',
 'beagle',
 'bloodhound',
 'bluetick',
 'black',
 'Walker_hound',
 'English_foxhound',
 'redbone',
 'borzoi',
 'Irish_wolfhound',
 'Italian_greyhound',
 'whippet',
 'Ibizan_hound',
 'Norwegian_elkhound',
 'otterhound',
 'Saluki',
 'Scottish_deerhound',
 'Weimaraner',
 'Staffordshire_bullterrier',
 'American_Staffordshire_terrier',
 'Bedlington_terrier',
 'Border_terrier',
 'Kerry_blue_terrier',
 'Irish_terrier',
 'Norfolk_terrier',
 'Norwich_terrier',
 'Yorkshire_terrier',
 'wire',
 'Lakeland_terrier',
 'Sealyham_terrier',
 'Airedale',
 'cairn',
 'Australian_terrier',
 'Dandie_Dinmont',
 'Boston_bull',
 'miniature_schnauzer',
 'giant_schnauzer',
 'standard_schnauzer',
 'Scotch_terrier',
 'Tibetan_terrier',
 'silky_terrier',
 'soft',
 'West_Highland_white_terrier',
 'Lhasa',
 'flat',
 'curly',
 'golden_retriever',


## Train, Validation, and Test Split

In [ ]:
lenTrain = int(lenImages * 0.7)
lenValidation = int(lenImages * 0.1)
lenTest = lenImages - lenTrain - lenValidation

assert lenTrain + lenValidation + lenTest == lenImages

train, validation, test = random_split(images, [lenTrain, lenValidation, lenTest], generator=torch.Generator().manual_seed(42))

In [ ]:
print(f'Train Size: {len(train)}')
print(f'Validation Size: {len(validation)}')
print(f'Test Size: {len(test)}')

Train Size: 14405
Validation Size: 2058
Test Size: 4117


## Transformation and Augmentation

In [ ]:
class CustomBreedDataset(Dataset):
    def __init__(self, dataset, transform=None):
        """
        Args:
            dataset (Dataset): Images
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
      image, label = self.dataset[idx]
      if self.transform:
        image = self.transform(image)  
        return image, label

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomRotation(degrees=45),
    transforms.RandomCrop(224, padding = 4, padding_mode = 'reflect'),
    transforms.RandomHorizontalFlip(p=0.3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
])

validation_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224,224)), 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
])

In [ ]:
train_dataset = CustomBreedDataset(train, train_transform)
validation_dataset = CustomBreedDataset(validation, validation_transform)
test_dataset = CustomBreedDataset(test, test_transform)

assert len(train_dataset) + len(validation_dataset) + len(test_dataset) == lenImages

## Creating Dataloaders

In [ ]:
batch_size = 32

# Speeding up the transfer from CPU to GPU by enabling pin_memory
use_gpu = torch.cuda.is_available()

train_dataloader = DataLoader(train_dataset, batch_size, shuffle=True, num_workers=2, pin_memory=use_gpu)
validation_dataloader = DataLoader(validation_dataset, batch_size*2, num_workers=2, pin_memory=use_gpu)
test_dataloader = DataLoader(test_dataset, batch_size*2, num_workers=2, pin_memory=use_gpu)

# Neural Network Architecture

In [ ]:
class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        img, targets = batch
        out = self(img)
        loss = F.nll_loss(out, targets)
        return loss
    
    def validation_step(self, batch):
        img, targets = batch
        out = self(img)
        loss = F.nll_loss(out, targets)
        _, preds = torch.max(out, dim=1)
        acc = torch.tensor(torch.sum(preds == targets).item() / len(preds))
        return {'val_acc':acc.detach(), 'val_loss':loss.detach()}
    
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()
        return {'val_loss':epoch_loss.item(), 'val_acc':epoch_acc.item()}
        
    def epoch_end(self, epoch, result):
        print("Epoch [{}] : train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(epoch, result["train_loss"], result["val_loss"], result["val_acc"]))

In [ ]:
class DogBreedClassificationCNN(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),

            nn.Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),

            nn.Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),

            nn.Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),

            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
        )

        self.avgpool = nn.AdaptiveAvgPool2d(7)

        self.classifier = nn.Sequential(
            nn.Linear(in_features=25088, out_features=4096, bias=True),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5, inplace=False),

            nn.Linear(in_features=4096, out_features=4096, bias=True),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5, inplace=False),

            nn.Linear(in_features=4096, out_features=120, bias=True),
            nn.LogSoftmax()
        )
    
    def forward(self, x):
      x = self.features(x)
      x = self.avgpool(x)
      x = x.view(-1, 25088)
      return self.classifier(x)

In [ ]:
model = DogBreedClassificationCNN()

In [ ]:
def to_device(data, device):
    if isinstance(data, (list, tuple)):
        return [to_device(d, device) for d in data]
    else:
        return data.to(device, non_blocking=True)

In [ ]:
class DeviceDataLoader:
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
    
    def __len__(self):
        return len(self.dl)
    
    def __iter__(self):
        for batch in self.dl:
            yield to_device(batch, self.device)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

train_dataloader = DeviceDataLoader(train_dataloader, device)
validation_dataloader = DeviceDataLoader(validation_dataloader, device)

to_device(model, device);

cuda


In [ ]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']
        
def fit_one_cycle(epochs, max_lr, model, train_loader, val_loader, weight_decay=0, grad_clip=None, opt_func = torch.optim.Adam):
    torch.cuda.empty_cache()
    history = []
    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs, steps_per_epoch=len(train_loader))
    
    for epoch in range(epochs):
        model.train()       
        train_losses = []
        lrs = []
        for batch in tqdm(train_loader):
            loss = model.training_step(batch)
            train_losses.append(loss)
            
            loss.backward()
            
            if grad_clip:
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)
                
            optimizer.step()
            optimizer.zero_grad()
            
            lrs.append(get_lr(optimizer))
            
            sched.step()
            
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['lrs'] = lrs
        model.epoch_end(epoch, result)
        history.append(result)

    return history

@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

In [ ]:
evaluate(model, validation_dataloader)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


{'val_acc': 0.01278409082442522, 'val_loss': 4.78729248046875}

In [ ]:
history = fit_one_cycle(8, 0.01, model, train_dataloader, validation_dataloader, 1e-4, 0.1, torch.optim.SGD)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)



Epoch [0] : train_loss: 4.7873, val_loss: 4.7855, val_acc: 0.0114



Epoch [1] : train_loss: 4.7847, val_loss: 4.7835, val_acc: 0.0114



Epoch [2] : train_loss: 4.7819, val_loss: 4.7828, val_acc: 0.0128



Epoch [3] : train_loss: 4.7804, val_loss: 4.7829, val_acc: 0.0128



Epoch [4] : train_loss: 4.7797, val_loss: 4.7831, val_acc: 0.0128



Epoch [5] : train_loss: 4.7793, val_loss: 4.7832, val_acc: 0.0128



Epoch [6] : train_loss: 4.7786, val_loss: 4.7832, val_acc: 0.0128



Epoch [7] : train_loss: 4.7782, val_loss: 4.7834, val_acc: 0.0128


In [ ]:
test_dataloader = DeviceDataLoader(test_dataloader, device)

result = evaluate(model, test_dataloader)
result

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


{'val_acc': 0.01225961558520794, 'val_loss': 4.785401821136475}

In [ ]:
# nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False),
#             nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False),

#             nn.Sequential(
#                 nn.Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False),
#                 nn.BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
#                 nn.BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False),
#                 nn.BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.ReLU(inplace=True),

#                 nn.Sequential(
#                     nn.Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False),
#                     nn.BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),       
#                 )

#                 nn.Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1), bias=False),
#                 nn.BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
#                 nn.BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False),
#                 nn.BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.ReLU(inplace=True),

#                 nn.Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1), bias=False),
#                 nn.BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
#                 nn.BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False),
#                 nn.BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.ReLU(inplace=True),
#             )

#             nn.Sequential(
#                 nn.Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1), bias=False),
#                 nn.BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False),
#                 nn.BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.Conv2d(256, 512, kernel_size=(1, 1), stride=(1, 1), bias=False),
#                 nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.ReLU(inplace=True),

#                 nn.Sequential(
#                     nn.Conv2d(256, 512, kernel_size=(1, 1), stride=(2, 2), bias=False),
#                     nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 )

#                 nn.Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1), bias=False),
#                 nn.BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
#                 nn.BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.Conv2d(256, 512, kernel_size=(1, 1), stride=(1, 1), bias=False),
#                 nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.ReLU(inplace=True),

#                 nn.Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1), bias=False),
#                 nn.BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
#                 nn.BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.Conv2d(256, 512, kernel_size=(1, 1), stride=(1, 1), bias=False),
#                 nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.ReLU(inplace=True),

#                 nn.Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1), bias=False),
#                 nn.BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
#                 nn.BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.Conv2d(256, 512, kernel_size=(1, 1), stride=(1, 1), bias=False),
#                 nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.ReLU(inplace=True),
#             )

#             nn.Sequential(
#                 nn.Conv2d(512, 512, kernel_size=(1, 1), stride=(1, 1), bias=False),
#                 nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False),
#                 nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.Conv2d(512, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False),
#                 nn.BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.ReLU(inplace=True),

#                 nn.Sequential(
#                   nn.Conv2d(512, 1024, kernel_size=(1, 1), stride=(2, 2), bias=False),
#                   nn.BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),       
#                 )

#                 nn.Conv2d(1024, 512, kernel_size=(1, 1), stride=(1, 1), bias=False),
#                 nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
#                 nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.Conv2d(512, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False),
#                 nn.BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.ReLU(inplace=True),

#                 nn.Conv2d(1024, 512, kernel_size=(1, 1), stride=(1, 1), bias=False),
#                 nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
#                 nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.Conv2d(512, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False),
#                 nn.BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.ReLU(inplace=True),

#                 nn.Conv2d(1024, 512, kernel_size=(1, 1), stride=(1, 1), bias=False),
#                 nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
#                 nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.Conv2d(512, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False),
#                 nn.BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.ReLU(inplace=True),

#                 nn.Conv2d(1024, 512, kernel_size=(1, 1), stride=(1, 1), bias=False),
#                 nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
#                 nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.Conv2d(512, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False),
#                 nn.BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.ReLU(inplace=True),

#                 nn.Conv2d(1024, 512, kernel_size=(1, 1), stride=(1, 1), bias=False),
#                 nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
#                 nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.Conv2d(512, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False),
#                 nn.BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.ReLU(inplace=True),
#             )

#             nn.Sequential(
#                 nn.Conv2d(1024, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False),
#                 nn.BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.Conv2d(1024, 1024, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False),
#                 nn.BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.Conv2d(1024, 2048, kernel_size=(1, 1), stride=(1, 1), bias=False),
#                 nn.BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.ReLU(inplace=True),

#                 nn.Sequential(
#                     nn.Conv2d(1024, 2048, kernel_size=(1, 1), stride=(2, 2), bias=False),
#                     nn.BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 )

#                 nn.Conv2d(2048, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False),
#                 nn.BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.Conv2d(1024, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
#                 nn.BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.Conv2d(1024, 2048, kernel_size=(1, 1), stride=(1, 1), bias=False),
#                 nn.BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.ReLU(inplace=True),

#                 nn.Conv2d(2048, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False),
#                 nn.BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.Conv2d(1024, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
#                 nn.BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.Conv2d(1024, 2048, kernel_size=(1, 1), stride=(1, 1), bias=False),
#                 nn.BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.ReLU(inplace=True),
#             )

#             nn.AdaptiveAvgPool2d(output_size=(1, 1)),
#             nn.Linear(in_features=2048, out_features=120, bias=True),
#             nn.LogSoftmax(dim=1),